## yfactor_necstdb.py 用可視化ツールの template です。
- 概要：
  - 全 cell を実行すると、20 IF の hot / cold の oneshot と y-factor が表示されます
- 特筆事項：
  - 20 IF 分のデータが存在しなくても、存在する IF データのみで plot 可能です
-  <font color="red">notebook 上で plot を行いたい場合：　</font>
  - 上から順番に実行していただければ、plot が表示されます
  - 画像を保存したくない場合は、「plt.savefig」 をコメントアウトしてください

### モジュールを import します。

In [1]:
import os
import sys
import time
from tqdm import tqdm
import numpy as np
import pandas as pd
import matplotlib
matplotlib.use('Agg')
matplotlib.rcParams['font.size'] = 12
matplotlib.rcParams['figure.facecolor'] = 'white'
import matplotlib.pyplot as plt

import warnings
warnings.filterwarnings('ignore')
if sys.argv[1] == 'xxx': pass
else: 
    %matplotlib inline

import necstdb

### database を読み込みます。

In [2]:
cwd = os.getcwd()
mode, date = cwd.split('/')[-2], cwd.split('/')[-1]

data_path_hot = '/media/usbdisk/data/{mode}/hot/{date}.necstdb/'.format(**locals())
data_path_cold = '/media/usbdisk/data/{mode}/cold/{date}.necstdb/'.format(**locals())

db_hot = necstdb.opendb(data_path_hot)
db_cold = necstdb.opendb(data_path_cold)

### hot, cold, y-factor, Trx を計算します。

In [3]:
IF_list = tqdm([
    'xffts_board01', 'xffts_board02',  'xffts_board03', 'xffts_board04',
    'xffts_board05', 'xffts_board06', 'xffts_board07', 'xffts_board08',
    'xffts_board09', 'xffts_board10', 'xffts_board11', 'xffts_board12',
    'xffts_board13', 'xffts_board14', 'xffts_board15', 'xffts_board16'
])
d_hot, d_cold = {}, {}
for IF in IF_list:
    IF_spec_name = IF
    try:
        d_hot[IF_spec_name] = np.array(db_hot.open_table(IF).read())
        d_cold[IF_spec_name] = np.array(db_cold.open_table(IF_spec_name).read())
        IF_list.set_description('[INFO] : {} DB is loaded.'.format(IF_spec_name))
    except:
        d_hot[IF_spec_name] = np.zeros((2, 2 ** 15 + 1)) # shape = (2, 32769)
        d_cold[IF_spec_name] = np.zeros((2, 2 ** 15 + 1))
        IF_list.set_description('[ALART] : {} DB is not exist.'.format(IF))

[ALART] : xffts_board13xffts_board14xffts_board15xffts_board16 DB is not exist.: 100%|██████████| 13/13 [00:00<00:00, 155.17it/s]


In [56]:
hot, cold = {}, {}
try:
    for key, val_hot, val_cold in zip(d_hot.keys(), d_hot.values(), d_cold.values()):
        hot[key]= np.delete(np.average(val_hot, axis=0), 0)
        cold[key]= np.delete(np.average(val_cold, axis=0), 0)
except: pass        
# memory freeing    
del(d_hot)
del(d_cold)

In [57]:
freq = np.linspace(0, 2000, 2**15 + 1) / 1e3
Thot, Tcold = 300., 77.
yfac, Trx = {}, {}
for key, val_hot, val_cold in zip(hot.keys(), hot.values(), cold.values()):
    yfac[key] = 10 * np.log10(val_hot / val_cold )
    Trx[key] = (Thot - Tcold) * yfac[key] / (yfac[key] - 1)

### yfactor を plot します。

In [ ]:
ncol = 5
nrow = 4
nax = ncol * nrow

figsize = (ncol * 4.0, nrow * 3.5)

matplotlib.rcParams['font.size'] = 12

fig = matplotlib.pyplot.figure(figsize=figsize)
ax = tqdm([fig.add_subplot(nrow, ncol, i+1) for i in range(nax)])

for _ax, beam, _hot, _cold, _yfac in zip(ax, hot.keys(), hot.values(), cold.values(), yfac.values()):
    ax.set_description('[INFO] : {} data is plotted'.format(beam.replace('xffts_spec_', '')))
    _ax_ = _ax.twinx()
    
    _ax.plot(freq, _cold, label='cold')
    _ax.plot(freq, _hot, label='hot')
    _ax.set_xlabel('frequency ( GHz )')
    _ax.set_yscale('log')
    _ax.set_ylabel('count')
    _ax.set_title(beam)
    _ax.grid()
    
    _ax_.plot(freq, _yfac, label='yfactor', c='g')
    _ax_.set_ylabel('Y-factor ( dB ) ')
    _ave_yfac = np.average(_yfac)
    _ax_.text(0.1, 0.1, 'Y-factor : \n     {:.2f} dB'.format(_ave_yfac), 
              color='darkolivegreen', fontweight='bold', transform=_ax.transAxes)
    
    h1, l1 = _ax.get_legend_handles_labels()
    h2, l2 = _ax_.get_legend_handles_labels()
    _ax_.legend(h1 + h2, l1 + l2, loc='lower right', fontsize=10)
    
plt.suptitle('{date} : {mode}'.format(**locals()), fontsize=20)
plt.subplots_adjust(wspace=0.2, hspace=0.25, bottom=0.05)
plt.tight_layout()
plt.subplots_adjust(top=0.935)


figname = '{date}_{mode}'.format(**locals())
plt.savefig('./{}.png'.format(figname))
print('info : Saved image. ( {}.png )'.format(figname))

[INFO] : 1r_l data is plotted: 100%|██████████| 20/20 [00:01<00:00, 16.27it/s]
